In [ ]:
# !pip install tensorboard tensorflow pandas

"""
If tensorboard is not installed (or other dependencies, such as tensorflow and pandas),
uncomment the command in top and re-run. This needs only to be run once in a Jupyter kernel.
"""

%load_ext tensorboard

from tensorflow.python.summary.summary_iterator import summary_iterator
import pandas as pd

In [ ]:
!tensorboard dev upload --logdir \
    '../logging'

In [ ]:
"""
Change the LOG_DIR argument to point to the correct directory, you may want to use an
absolute path if you run into issues.
"""
# !!kill 22140
%tensorboard --logdir ./logging

In [ ]:
def logs_to_pandas(path: str) -> pd.DataFrame:
    """convert single tensorflow log file to pandas DataFrame
    Parameters
    ----------
    path : str
        path to tensorflow log file
    Returns
    -------
    pd.DataFrame
        converted dataframe
    """

    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": [], "wall_time": []})
    try:
        event_acc = summary_iterator(path)
        for event in list(event_acc)[1:]:
            step, wall_time = event.step, pd.to_datetime(event.wall_time, unit='s')
            simple_extractor = [{"metric": v.tag, "value": v.simple_value, "step": step, 'wall_time': wall_time} for v in event.summary.value]
            event_r = pd.DataFrame(simple_extractor)
            runlog_data = pd.concat([runlog_data, event_r])
    #Dirty catch of DataLossError
    except Exception as e:
        raise(e)
        print("Event file possibly corrupt: {}".format(path))
        print(e)
    return runlog_data

In [ ]:
from pathlib import Path
import subprocess
import yaml

# creates dictionary: <id, configmap>
job_ids = [i for i in subprocess.getoutput("kubectl get configmap --all-namespaces").split(" ") if "master" in i]
dict_configmaps =  {}
for j in job_ids:
    print("***" * 5)
    print(j)
    config_map = subprocess.getoutput(f"kubectl get configmaps {j} -o yaml -n test")
    dict_configmaps[j.split("master-")[1][:-2]] = config_map

# creates dictionary: <id, event log dataframe>
dict_log_results = {}
for path in Path('./logging').rglob('*events.out*'):
    path_iid = path.name.split("trainjob-")[1].split("-master")[0]
    dict_log_results[path_iid] = logs_to_pandas(f"./{path}")
    
print("\n finished creating dictionaries")
print("keys configmaps: ", list(dict_configmaps.keys()))
print("keys log results: ", list(dict_log_results.keys()))